# Exercise 4: Recognising handwritten digits

### The digits dataset:
Use the Scikit-learn digits dataset (`from sklearn.datasets import load_digits`).
The data consist of images from handwritten digits and contains 250 samples from 44 writers.

This data is stored in the `.data` member, which is an `n_samples`, `n_features` array.

The target values are stored in the `.target` member, which is an `n_samples` array of integers.

## Exercises

1. Load the digits dataset and print the shape of the data and the target.
2. Display the ten first images and the training set and the corresponding target values.
3. Split the data set into a training and test set and train a (multinomial) logistic regression classifier on the dataset and evaluate its accuracy.
4. Write some code that draw one random image from the test set, displays this image, and print the predicted value of the digit.
5. Train some alternative classifiers and compare their accuracy to the logistic regression classifier. Here are some suggested models:
    a. Decision tree
    b. K-nearest neighbors
    c. Naive Bayes
    d. Support vector machine (not covered in this course)
6. Try to improve the accuracy of some classifiers by tuning the hyperparameters such as `max_depth` for the decision tree or `n_neighbors` for the KNN classifier.
7. (Optional) If you want to experiment with a larger dataset, try the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). This dataset contains 70,000 images of handwritten digits, and is a good starting point for more advanced image classification tasks.